In [ ]:
import os
import glob
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append("../../../")

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from helpers.split import tag_label_feature_split, make_train_test_split
from helpers.assess import make_confusion_matrix, make_classification_report

In [ ]:
def logistic_regression_experiment(
    dataset, optimizer="lbfgs", max_iteration=1000, verbose=0
):

    # load data and separate label strings from features,
    # then encode label strings

    print(f"\n\nDataset: {dataset}\n")    

    df = pd.read_pickle(dataset)
    _, (y, le) , X = tag_label_feature_split(df,label_format='encoded')

    # divide into train and test data sets
    X_train_std, X_test_std, y_train, y_test = make_train_test_split(
        X, y, test_size=0.2, random_state=10, stratify=y,x_scaler="standard"
    )

    model = LogisticRegression(
        C=100.0,
        solver=optimizer,
        multi_class="multinomial",
        class_weight="balanced",
        max_iter=max_iteration,
        verbose=verbose,
    )

    model.fit(X_train_std, y_train)

    predictions = model.predict(X_test_std)
    train_accuracy = model.score(X_train_std, y_train)
    test_accuracy = model.score(X_test_std, y_test)
 
    print(f"Train: {train_accuracy = :f}\n")
    make_classification_report(y_train,model=model,x=X_train_std,label_encoder=le,print_report=True)
    print(f"Test: {test_accuracy = :f}\n")
    make_classification_report(y_test, y_pred=predictions, digits=4,label_encoder=le,print_report=True)
    
    name = os.path.basename(dataset).split(".")[0]
    make_confusion_matrix(
        y_test,
        y_pred=predictions,
        label_encoder=le,
        title=f"{name} test (row normalized)"
    )

    return model

In [ ]:
dataset_folder = "../../../datasets/"
datasets = sorted([name for name in glob.glob(dataset_folder + "*.pickle")])
for dataset in datasets:
    logistic_regression_experiment (dataset)